## 5 MEMORY
### 5.0 ConversationBufferMemory

전체 메모리를 저장하는 방법

단점 : 비용이 많이 듬

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input":"Hi!"},{"output":"How are you?"})

memory.load_memory_variables({})

### ConversationBufferWindowMemory

최근 메모리만을 저장하는 방법

단점 : 최근 대화에만 집중함

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

add_message(1, 1)

In [ ]:
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)

In [ ]:
add_message(5,5)

In [ ]:
memory.load_memory_variables({})

## 5.2 ConversationSummaryMemory 

llm을 사용하여 요약하여 메모리에 저장

장점 : 매우 긴 대화에서 간편하게 메모리를 저장할 수 있음

단점 : 메모리를 실행하기 위해 llm을 동작해야함


In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo-0125",
    temperature=0.1,
)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
add_message("South Kddorea is so pretty", "I wish I could go!!!")

In [ ]:
get_history()

### 5.3 ConversationSummaryBufferMemory

Conversation Summary Memory + Conversation Buffer Memory

메모리에 보내온 메시지의 수를 저장

최근 메시지는 그대로 저장

리미트에 다달한 메시지는 요약하여 저장함


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    model_name = "gpt-3.5-turbo-0125",
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
get_history()

In [ ]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [ ]:
get_history()

In [ ]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [ ]:
get_history()

### 5.4 ConversationKGMemory 

Conversation Knowledge Graph Memory

Knowledge Graph를 뽑아내어 요약

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("He is Hyoin, he from Korean", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"input": "who is Hyoin"})

In [ ]:
add_message("He likes hamberger.", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"input": "What does Hyoin like?"})

### 5.5 Memory on LLMChain 

메모리와 LLM을 연결하여 작동

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",   
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=80,
    memory_key="chat_history",
)


template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")

In [ ]:
chain.predict(question="where am I")

In [ ]:
chain.predict(question="I'm 27")

In [ ]:
chain.predict(question = "How old am I?")

### 5.6 Chat Based Memory

실제 채팅이 가능한 메모리로 바꿈

기존에 코드는 메모리를 string으로 저장함

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",   
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages = True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}")
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")
chain.predict(question="I'm 27")

In [ ]:
chain.predict(question="Qize Time! What is my name? And How old am I?")

### 5.7 LCEL Based Memory

LangChain Expression Language

메모리에 쉽게 추가하는 방법

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",   
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=80,
    return_messages = True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

# invoke_chain("My name is eggsy")

In [ ]:
invoke_chain("My name is eggsy")